In [ ]:
import pandas as pd
stores = pd.read_csv("./data/lifestylegroup_final_tmp.csv", engine="python")

In [ ]:
stores

In [ ]:

import json
import pandas as pd
import os
import shutil



DATA_DIR = "./data"
DATA_FILE = os.path.join(DATA_DIR, "data.json")
DUMP_FILE = os.path.join(DATA_DIR, "dump.pkl")


store_columns = (
    "id",  # 음식점 고유번호
    "store_name",  # 음식점 이름
    "branch",  # 음식점 지점 여부
    "area",  # 음식점 위치
    "tel",  # 음식점 번호
    "address",  # 음식점 주소
    "latitude",  # 음식점 위도
    "longitude",  # 음식점 경도
    "category",  # 음식점 카테고리
)

review_columns = (
    "id",  # 리뷰 고유번호
    "store",  # 음식점 고유번호 (FK)
    "user",  # 유저 고유번호 (Fk)
    "score",  # 평점
    "content",  # 리뷰 내용
    "reg_time",  # 리뷰 등록 시간
)

menu_columns = (
    "id",  # 메뉴 고유번호
    "store",  # 음식점 고유번호 (FK)
    "menu_name", # 메뉴 이름
    "price",  # 메뉴 가격
)


user_columns = (
    "id",  # 유저 고유번호
    "gender",  # 유저 성별
    "age",  # 유저 나이
)



In [ ]:

def import_data(data_path=DATA_FILE):
    """
    Req. 1-1-1 음식점 데이터 파일을 읽어서 Pandas DataFrame 형태로 저장합니다
    """


    try:
        with open(data_path, encoding="utf-8") as f:
            data = json.loads(f.read())
    except FileNotFoundError as e:
        print(f"`{data_path}` 가 존재하지 않습니다.")
        exit(1)
#     print('=======================json_loads=====================')
#     test = pd.DataFrame(data=data)
#     print(test['review_list'])
#     print('=======================json_loads=====================')

    stores = []  # 음식점 테이블
    reviews = []  # 리뷰 테이블
    menus = [] # 메뉴 테이블
    users = [] # 유저 테이블
    
    idx = 0
    for d in data:

        categories = [c["category"] for c in d["category_list"]]
        stores.append(
            [
                d["id"],
                d["name"],
                d["branch"],
                d["area"],
                d["tel"],
                d["address"],
                d["latitude"],
                d["longitude"],
                "|".join(categories),
            ]
        )

        for review in d["review_list"]:
            r = review["review_info"]
            u = review["writer_info"]

            reviews.append(
                [r["id"], d["id"], u["id"], r["score"], r["content"], r["reg_time"]]
            )
            
            users.append(
                [u["id"], u["gender"], 2021-int(u["born_year"])]
            )
        
        for m in d["menu_list"]:
            idx += 1
            menus.append(
                [str(idx), d["id"], m["menu"], m["price"]]
            )
    store_frame = pd.DataFrame(data=stores, columns=store_columns)
    review_frame = pd.DataFrame(data=reviews, columns=review_columns)
    user_frame = pd.DataFrame(data=users, columns=user_columns)
    menu_frame = pd.DataFrame(data=menus, columns=menu_columns)
    

    return {"stores": store_frame, "reviews": review_frame, "users":user_frame, "menus":menu_frame,}


def dump_dataframes(dataframes):
    pd.to_pickle(dataframes, DUMP_FILE)


def load_dataframes():
    return pd.read_pickle(DUMP_FILE)

def main():

    print("[*] Parsing data...")
    data = import_data()
    print("[+] Done")

    print("[*] Dumping data...")
    dump_dataframes(data)
    print("[+] Done\n")

    data = load_dataframes()

    term_w = shutil.get_terminal_size()[0] - 1
    separater = "-" * term_w

    print("[음식점]")
    print(f"{separater}\n")
    print(data["stores"].head(20))
    print(f"\n{separater}\n\n")

    print("[리뷰]")
    print(f"{separater}\n")
    print(data["reviews"].head(20))
    print(f"\n{separater}\n\n")

    print("[메뉴]")
    print(f"{separater}\n")
    print(data["menus"].head(20))
    print(f"\n{separater}\n\n")
        
    
    print("[유저]")
    print(f"{separater}\n")
    print(data["users"].head(20))
    print(f"\n{separater}\n\n")

    
    

if __name__ == "__main__":
    data = load_dataframes()


In [ ]:
users = data["users"]
reviews = data["reviews"]

In [ ]:
stores_reviews = pd.merge(
reviews, users, left_on="user", right_on="id"
)

In [ ]:
stores_reviews = stores_reviews.drop_duplicates()
stores_reviews

In [ ]:
df = pd.merge(
stores, stores_reviews, left_on="id", right_on="store"
)

In [ ]:
df.columns # big_cate 있는지 확인
df

In [ ]:
df = df[["user", "big_cate","score"]] # big_cate 이거를 detail, map 등 라이프스타일 적용해줘요!

In [ ]:
# 테이블을 딕셔너리로 만들기
def recur_dict(frame):
    if len(frame.columns) == 1:
        if frame.values.size == 1 : return frame.values[0][0]
        return frame.values.squeeze()
    grouped = frame.groupby(frame.columns[0])
    d = {k : recur_dict(g.iloc[:, 1:]) for k, g in grouped}
    return d

In [ ]:
df_to_dict = recur_dict(df)

In [ ]:
df_to_dict

In [ ]:
# 사용자 목록, 음식점 목록 리스트에 담기
name_list = [] # 사용자 목록 # 중복 불가
store_set = set() # 화장품 목록을 담을 set # 중복 가능

# 사용자 수만큼 반복함
for user_key in df_to_dict:
    name_list.append(user_key)
    # 현재 사용자가 리뷰를 남긴 가게 목록을 set에 담는다.
    for store_key in df_to_dict[user_key]:
        store_set.add(store_key)


In [ ]:
store_set

In [ ]:
store_list = list(store_set)

In [ ]:
# CF 추천시스템에 사용할 rating 추천 딕셔너리 만들기(유저번호, 가게이름, 평점, 카테고리)

In [ ]:
rating_dic = {
    'user'  :[],
    'store_name':[],
     'rating':[],
}
# 사용자 수 만큼 반복하는 알고리즘
for name_key in df_to_dict:
    for store_key in df_to_dict[name_key]:
        a1  = name_list.index(name_key)
        a2 = store_list.index(store_key)
        a3 = df_to_dict[name_key][store_key]
        
        rating_dic["user"].append(a1)
        rating_dic["store_name"].append(a2)
        rating_dic["rating"].append(a3)

In [ ]:
# 사용자 - 가게 - 평점 출력 => 나중에는 가게 말고 big 카테고리 기준으로 나눠서 카테고리 나눠볼까함
# 숫자가 같아야 함 세개가
print(len(rating_dic["user"]))
print(len(rating_dic["store_name"]))
print(len(rating_dic["rating"]))

In [ ]:
# numpy의 array 형태이므로 중간에 배열로 들어감 -> tolist 해서 평균값으로 대체해야

temp = [] #  63번쨰
for i in range(1327): # 위에 프린트로 출력한 len으로 바꿔야
#     print(i)
    temp.append(rating_dic["rating"][i].tolist())


In [ ]:
for i in range(1327): # 위에 프린트로 출력한 len으로 바꿔야
    if type(temp[i]) == list:
        a = 0
        l = len(temp[i])
#         print(l)
        for j in range(l):
            a += temp[i][j]
        a = a / l
        temp[i] = a
        

In [ ]:
rating_dic["rating"] = temp

In [ ]:
print(len(rating_dic["user"]))
print(len(rating_dic["store_name"]))
print(len(rating_dic["rating"]))

In [ ]:
# 데이터 셋 만들기
df = pd.DataFrame(rating_dic)
df

In [ ]:
# 딕셔너리 바탕으로 사용자 이름 

In [ ]:
import surprise
# 데이터를 읽어들이는 객체 생성(rating_scale=0~5)
reader = surprise.Reader(rating_scale=(0, 5))
reader

In [ ]:
import surprise
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
# surprise에서 사용할 데이터 셋을 구성하는데 필요한 이름
# 데이터가 저장되어 있는 딕셔너리의 컬럼 이름
# 첫번째 -> user, 두번째 -> store, 세번째 -> rating
col_list = ["user", "store_name", "rating"]
data = surprise.Dataset.load_from_df(df[col_list], reader)
# print(col_list)
print(data)
# cross_validate(NormalPredictor(), data, cv=3)

In [ ]:
# 학습
trainset = data.build_full_trainset()
option = {"name":"pearson"} # 피어슨 유사도 사용
algo = surprise.KNNBasic(sim_options=option) #surprise.KNNBasic사용
algo.fit(trainset)
# surprise.evaluate(algo, data)

In [ ]:
# print(len(name_list)) 
print(name_list) # 유저 번호 리스트 - 여기에 있는 번호로 넣어야


In [ ]:
for i in range(len(name_list)):
    name_list[i] = str(name_list[i])

In [ ]:
# 최종적으로 예측하는 CF 추천시스템
who = int(input("유저 번호를 입력하세요"))
print("\n")
who = str(who)
index = name_list.index(who)
print(index) # 인덱스인것주의
print("=============")
result = algo.get_neighbors(index, k=5) # k 조정
print("당신과 비슷한 입맛을 가진 유저는? : ", result) # 입맛이 비슷한 열명의 사용자 번호가 아니고, 인덱스인듯!
print("\n")

# df : rating한 df 
# user에게 추천하는 음식점
print("추천 드리는 카테고리 : ")
for r1 in result:
    max_rating=data.df[data.df["user"]==r1]["rating"].max()
    # 점수가 비등비등해서 max라고 해도 눈에 띄게 잘 나뉘지는 않음. 그래서 최고점 받은애들만 선별
#     store_id = data.df[(data.df["rating"]==max_rating)&(data.df["user"]==r1)]["store_name"].values
    store_id = data.df[(data.df["rating"]==5)&(data.df["user"]==r1)]["store_name"].values
#     print(store_id)

    # 추천해주는 가게들 목록
    for store_item in store_id:
        print(store_list[store_item])
